### Investigate final_score_unweighted has 16,582 zero values

In [ ]:
%run '../lib/libraries.ipynb'

In [ ]:
dfFlt = load_df('dfFlt')
types = get_var_types()

#### Create zero value dataframe

In [ ]:
zeroDf = dfFlt.filter(F.col('final_score_unweighted') == 0)

#### Count zero values

In [ ]:
zeroDf.count()

- 16,582 zero values

In [ ]:
distinct_val(zeroDf)

- No additional values

###### Count observations with no response

In [ ]:
zeroDf.filter(F.col('assessment_item_response_id').isNull()).count()

- 13,715

###### Exclude observations with no response

In [ ]:
noResponseDf = zeroDf.filter(~F.col('assessment_item_response_id').isNull())

###### Remaining count

In [ ]:
noResponseDf.count()

In [ ]:
- 2867

###### Are the remaining zero scores valid

In [ ]:
pdDf = add_zero_raw_score_var(noResponseDf).toPandas()

crosstab_percent( pd.crosstab(pdDf.zero_score, pdDf.response_correctness) )

- Remaining observations
  - raw_score = 0 (zero_score = Yes)
    - response_correctnss
      - 1% [unassigned]
      - 99% incorrect
  - raw_score > 0 (zero_score = No)
    - response_correctnss
      - 96% incorrect
      - 4% partially_correct

##### Conclusion:
- No further action needed
- The 16,582 zero values correct

In [ ]:
finish_todo('Investigate final_score_unweighted has 16,582 zero values')